# PM 5 Task H POD Truncation - match Demircan


In [ ]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
from scipy.sparse.linalg import eigs
import time

## Full Order Models

Inputs

In [ ]:
from eval_u_Sonar import *

waveforms_20Hz = {
    # Continuous
    'Continuous (training)': eval_u_Sonar_20_const,
    'Continuous (0.5x amp)': eval_u_20_const_half_amp,
    'Continuous (2x amp)': eval_u_20_const_double_amp,
    
    # Pulses
    'Pulse (short, 2 cycles)': eval_u_20_pulse_short,
    'Pulse (long, 8 cycles)': eval_u_20_pulse_long,
    'Pulse (Hann window)': eval_u_20_pulse_hann,
    'Pulse (rectangular)': eval_u_20_pulse_rect,
    
    # Modulated
    'AM (0.5 Hz mod)': eval_u_20_AM_slow,
    'AM (2 Hz mod)': eval_u_20_AM_fast,
    'FM (±2 Hz dev)': eval_u_20_FM,
    
    # Chirps
    'Chirp (15→25 Hz)': eval_u_chirp_up,
    'Chirp (25→15 Hz)': eval_u_chirp_down,
    'Chirp (19→21 Hz)': eval_u_chirp_narrow,
    
    # Bursts
    'Burst (on/off)': eval_u_20_burst,
    'Burst (fade)': eval_u_20_burst_fade,
    'Double pulse': eval_u_20_double_pulse,
    
    # Multi-frequency (boundary cases)
    '20 Hz + 40 Hz harmonic': eval_u_20_plus_harmonic,
    '20 Hz + 10 Hz subharmonic': eval_u_20_plus_subharmonic,
}

waveforms_out_of_band = {
    '100 Hz (out of band)': eval_u_100_const,
    '5 Hz (out of band)': eval_u_5_const,
}

# All waveforms
all_waveforms = {**waveforms_20Hz, **waveforms_out_of_band}

In [ ]:
from setup_sonar_model import setup_sonar_model, print_model_info

model = setup_sonar_model(
    Nx=201,
    Nz=51,
    Lx=2000,
    Lz=500,
    f0=20,
    source_position="center",
    hydrophone_config="horizontal",
    eval_u=eval_u_Sonar_20_const,
)

# Print info
print_model_info(model)

# Extract what you need
p = model['p']
x_start = model['x_start']
t_sim = model['t_stop']
max_dt_FE = model['max_dt_FE']
eval_u_scaled = model['eval_u_scaled']
eval_f = model['eval_f']
f0 = model['f0']

A = p['A']
B = p['B']

Nx, Nz = model['Nx'], model['Nz']
Lx, Lz = model['Lx'], model['Lz']
N = Nx * Nz
n = 2 * N

print(f"\nState dimension: 2N = {n:,}")

# Build C matrix for hydrophone outputs
hydro = model['hydrophones']
n_phones = hydro['n_phones']

if 'z_pos' in hydro:
    z_idx = hydro['z_pos']
    x_indices = hydro['x_indices']
    C = np.zeros((n_phones, n))
    for i, x_idx in enumerate(x_indices):
        obs_idx = N + x_idx * Nz + z_idx
        C[i, obs_idx] = 1.0
else:
    x_indices = hydro['x_indices']
    z_indices = hydro['z_indices']
    C = np.zeros((n_phones, n))
    for i, (x_idx, z_idx) in enumerate(zip(x_indices, z_indices)):
        obs_idx = N + x_idx * Nz + z_idx
        C[i, obs_idx] = 1.0

In [ ]:
from scipy.sparse import eye
from scipy.sparse.linalg import splu
from scipy.linalg import lu_factor, lu_solve
from sklearn.decomposition import TruncatedSVD
import time

dx, dz = model['dx'], model['dz']
dt = max_dt_FE * 0.5

def run_full_order(eval_u_func, save_snapshots=False):
    """Run full-order trapezoidal simulation"""
    
    def eval_u_scaled(t):
        return dx * dz * eval_u_func(t)
    
    num_steps = int(np.ceil(t_sim / dt))
    
    I_sparse = eye(n, format='csr')
    LHS = I_sparse - (dt/2) * A
    RHS_mat = I_sparse + (dt/2) * A
    LU = splu(LHS.tocsc())
    
    t_full = np.zeros(num_steps + 1)
    y_full = np.zeros((num_steps + 1, n_phones))
    if save_snapshots:
        X_full = np.zeros((n, num_steps + 1))
    
    x_curr = x_start.flatten().copy()
    B_dense = B.toarray().flatten()
    
    t_full[0] = 0.0
    y_full[0] = C @ x_curr
    if save_snapshots:
        X_full[:, 0] = x_curr
    
    t0 = time.perf_counter()
    for i in range(1, num_steps + 1):
        t_prev = t_full[i-1]
        t_curr = t_prev + dt
        
        u_prev = eval_u_scaled(t_prev)
        u_curr = eval_u_scaled(t_curr)
        
        rhs = RHS_mat @ x_curr + (dt/2) * B_dense * (u_prev + u_curr)
        x_curr = LU.solve(rhs)
        
        t_full[i] = t_curr
        y_full[i] = C @ x_curr
        if save_snapshots:
            X_full[:, i] = x_curr
    
    full_time = time.perf_counter() - t0
    
    if save_snapshots:
        return t_full, y_full, X_full, full_time
    return t_full, y_full, full_time

Train on 20 Hz constant

In [ ]:
print("Running full-order (training signal)...")
t_full_train, y_full_train, X_full, full_time_train = run_full_order(
    eval_u_Sonar_20_const, save_snapshots=True
)
print(f"Full-order time: {full_time_train:.2f}s")
print(f"Snapshots: {X_full.shape}")

# SVD
skip = 5
X_sub = X_full[:, ::skip]
k = min(100, X_sub.shape[1] - 1)

print(f"\nComputing SVD (k={k})...")
t0 = time.perf_counter()
svd = TruncatedSVD(n_components=k, algorithm='randomized', n_iter=5, random_state=42)
svd.fit(X_sub.T)
U = svd.components_.T
S = svd.singular_values_
svd_time = time.perf_counter() - t0
print(f"SVD time: {svd_time:.2f}s")

cumulative_energy = np.cumsum(S**2) / np.sum(S**2)

# Find stable q values
print("Finding stable q values...")
stable_qs = []
for q in range(1, k + 1):
    Phi_q = U[:, :q]
    A_pod_q = Phi_q.T @ (A @ Phi_q)
    max_re = np.real(np.linalg.eigvals(A_pod_q)).max()
    if max_re <= 0:
        stable_qs.append(q)

print(f"Stable q: {len(stable_qs)} found, max = {max(stable_qs)}")

# Build ROM
q_pod = max(stable_qs)
Phi = U[:, :q_pod]
A_pod = Phi.T @ (A @ Phi)
B_pod = Phi.T @ B.toarray()
C_pod = C @ Phi
x0_pod = Phi.T @ x_start

print(f"ROM: q={q_pod}, energy={cumulative_energy[q_pod-1]*100:.2f}%")

In [ ]:
def simulate_rom(eval_u_func):
    """Run ROM trapezoidal simulation"""
    
    def eval_u_scaled(t):
        return dx * dz * eval_u_func(t)
    
    t = np.arange(0, t_sim + dt, dt)
    n_steps = len(t)
    
    x = np.zeros((n_steps, q_pod))
    y = np.zeros((n_steps, n_phones))
    
    x_curr = x0_pod.flatten().copy()
    B_flat = B_pod.flatten()
    
    I = np.eye(q_pod)
    LHS = I - (dt/2) * A_pod
    RHS_mat = I + (dt/2) * A_pod
    LU = lu_factor(LHS)
    
    x[0] = x_curr
    y[0] = C_pod @ x_curr
    
    t0 = time.perf_counter()
    for i in range(1, n_steps):
        u_prev = eval_u_scaled(t[i-1])
        u_curr = eval_u_scaled(t[i])
        
        rhs = RHS_mat @ x_curr + (dt/2) * B_flat * (u_prev + u_curr)
        x_curr = lu_solve(LU, rhs)
        
        x[i] = x_curr
        y[i] = C_pod @ x_curr
    
    rom_time = time.perf_counter() - t0
    return t, y, x, rom_time

In [ ]:
results = {}
phone_idx = 2  # H3 (middle hydrophone)

print("="*70)
print("WAVEFORM SWEEP: ROM vs Full-Order")
print("="*70)

for name, u_func in all_waveforms.items():
    print(f"\n{name}...", end=" ")
    
    # ROM
    t_rom, y_rom, x_rom, rom_time = simulate_rom(u_func)
    
    # Full-order
    t_fo, y_fo, fo_time = run_full_order(u_func, save_snapshots=False)
    
    # Error at H3
    y_rom_interp = np.interp(t_fo, t_rom, y_rom[:, phone_idx])
    rel_error = np.linalg.norm(y_fo[:, phone_idx] - y_rom_interp) / \
                (np.linalg.norm(y_fo[:, phone_idx]) + 1e-12) * 100
    
    speedup = fo_time / rom_time
    
    results[name] = {
        't_rom': t_rom, 'y_rom': y_rom,
        't_fo': t_fo, 'y_fo': y_fo,
        'rom_time': rom_time, 'fo_time': fo_time,
        'error': rel_error, 'speedup': speedup
    }
    
    status = "✓" if rel_error < 5 else "⚠" if rel_error < 20 else "✗"
    print(f"{status} Error: {rel_error:.2f}% | Speedup: {speedup:.0f}x")

print("\n" + "="*70)

Run full order for all waveforms

In [ ]:
full_order_results = {}

print("="*70)
print("RUNNING FULL-ORDER FOR ALL WAVEFORMS")
print("="*70)

for name, u_func in all_waveforms.items():
    print(f"{name}...", end=" ")
    
    t_fo, y_fo, fo_time = run_full_order(u_func, save_snapshots=False)
    
    full_order_results[name] = {
        't': t_fo,
        'y': y_fo,
        'time': fo_time
    }
    
    print(f"Done in {fo_time:.2f}s")

print("\n" + "="*70)
total_fo_time = sum(r['time'] for r in full_order_results.values())
print(f"Total full-order time: {total_fo_time:.1f}s")

ROM for all waveforms and compare

In [ ]:
results = {}
phone_idx = 4  # H3 (middle hydrophone)

print("="*70)
print("RUNNING ROM AND COMPARING TO FULL-ORDER")
print("="*70)

for name, u_func in all_waveforms.items():
    print(f"{name}...", end=" ")
    
    # ROM simulation
    t_rom, y_rom, x_rom, rom_time = simulate_rom(u_func)
    
    # Get full-order from saved results
    fo = full_order_results[name]
    t_fo, y_fo, fo_time = fo['t'], fo['y'], fo['time']
    
    # Error at H3
    y_rom_interp = np.interp(t_fo, t_rom, y_rom[:, phone_idx])
    rel_error = np.linalg.norm(y_fo[:, phone_idx] - y_rom_interp) / \
                (np.linalg.norm(y_fo[:, phone_idx]) + 1e-12) * 100
    
    speedup = fo_time / rom_time
    
    results[name] = {
        't_rom': t_rom, 'y_rom': y_rom, 'x_rom': x_rom,
        't_fo': t_fo, 'y_fo': y_fo,
        'rom_time': rom_time, 'fo_time': fo_time,
        'error': rel_error, 'speedup': speedup
    }
    
    status = "✓" if rel_error < 5 else "⚠" if rel_error < 20 else "✗"
    print(f"{status} Error: {rel_error:.2f}% | Speedup: {speedup:.0f}x")

print("\n" + "="*70)
total_rom_time = sum(r['rom_time'] for r in results.values())
print(f"Total ROM time: {total_rom_time:.3f}s")
print(f"Total speedup: {total_fo_time/total_rom_time:.0f}x")

Summary Table

In [ ]:
print("\n" + "="*85)
print(f"{'Waveform':<32} {'Full (ms)':<12} {'ROM (ms)':<12} {'Speedup':<10} {'Error (%)':<10}")
print("="*85)

# Sort by error
sorted_results = sorted(results.items(), key=lambda x: x[1]['error'])

for name, r in sorted_results:
    status = "✓" if r['error'] < 5 else "⚠" if r['error'] < 20 else "✗"
    print(f"{status} {name:<30} {r['fo_time']*1000:<12.1f} {r['rom_time']*1000:<12.2f} "
          f"{r['speedup']:<10.0f}x {r['error']:<10.2f}")

print("="*85)

# Summary
good = [n for n, r in results.items() if r['error'] < 5]
marginal = [n for n, r in results.items() if 5 <= r['error'] < 20]
failed = [n for n, r in results.items() if r['error'] >= 20]

print(f"\n✓ Good (<5% error): {len(good)}")
print(f"⚠ Marginal (5-20% error): {len(marginal)}")
print(f"✗ Failed (>20% error): {len(failed)}")

if failed:
    print(f"\nFailed waveforms: {failed}")

Visualize

In [ ]:
import matplotlib.pyplot as plt

# Select representative waveforms to plot
plot_list = [
    'Continuous (training)',
    'Pulse (Hann window)', 
    'Chirp (15→25 Hz)',
    'AM (2 Hz mod)',
    'Burst (fade)',
    '20 Hz + 40 Hz harmonic',
    '100 Hz (out of band)',
    '5 Hz (out of band)',
]

# Filter to existing
plot_list = [p for p in plot_list if p in results]

fig, axes = plt.subplots(len(plot_list), 2, figsize=(14, 2.5*len(plot_list)))

for i, name in enumerate(plot_list):
    r = results[name]
    
    # Left: ROM vs Full-order
    axes[i, 0].plot(r['t_fo']*1000, r['y_fo'][:, phone_idx], 'b-', lw=1.5, label='Full-order')
    axes[i, 0].plot(r['t_rom']*1000, r['y_rom'][:, phone_idx], 'r--', lw=1.5, label='ROM')
    axes[i, 0].set_ylabel('Pressure (Pa)', fontsize=9)
    axes[i, 0].set_title(f'{name}', fontsize=10, loc='left')
    axes[i, 0].legend(loc='upper right', fontsize=8)
    axes[i, 0].grid(True, alpha=0.3)
    
    # Right: Error
    y_rom_interp = np.interp(r['t_fo'], r['t_rom'], r['y_rom'][:, phone_idx])
    error = r['y_fo'][:, phone_idx] - y_rom_interp
    
    color = 'green' if r['error'] < 5 else 'orange' if r['error'] < 20 else 'red'
    axes[i, 1].plot(r['t_fo']*1000, error, '-', color=color, lw=1)
    axes[i, 1].set_ylabel('Error (Pa)', fontsize=9)
    axes[i, 1].set_title(f"Error: {r['error']:.2f}%", fontsize=10, loc='left')
    axes[i, 1].grid(True, alpha=0.3)

axes[-1, 0].set_xlabel('Time (ms)')
axes[-1, 1].set_xlabel('Time (ms)')

plt.suptitle(f'ROM (q={q_pod}, trained on 20 Hz const) vs Full-Order', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

names = [n for n, _ in sorted_results]
errors = [r['error'] for _, r in sorted_results]
colors = ['green' if e < 5 else 'orange' if e < 20 else 'red' for e in errors]

bars = ax.barh(range(len(names)), errors, color=colors, alpha=0.7)
ax.set_yticks(range(len(names)))
ax.set_yticklabels(names, fontsize=9)
ax.set_xlabel('Relative Error (%)')
ax.set_title(f'ROM Generalization Error (q={q_pod}, trained on Continuous 20 Hz)')

ax.axvline(5, color='green', linestyle='--', alpha=0.7, label='Good (<5%)')
ax.axvline(20, color='orange', linestyle='--', alpha=0.7, label='Marginal (<20%)')
ax.legend(loc='lower right')
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

In [ ]:
%matplotlib qt

import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button
import numpy as np

# Select 5 waveforms to display
display_waveforms = [
    'Continuous (training)',
    'Pulse (Hann window)',
    'Chirp (15→25 Hz)',
    'AM (2 Hz mod)',
]

display_waveforms = [w for w in display_waveforms if w in results]
n_waveforms = len(display_waveforms)

# Setup figure: one column per waveform
fig, axes = plt.subplots(1, n_waveforms, figsize=(4 * n_waveforms, 4))
plt.subplots_adjust(bottom=0.15, wspace=0.3)

# Limit to 800ms
t_rom = results[display_waveforms[0]]['t_rom']
t_max = 0.8
max_idx = np.searchsorted(t_rom, t_max)
max_idx = min(max_idx, len(t_rom) - 1)

n_frames = min(200, max_idx)
frame_indices = np.linspace(0, max_idx, n_frames, dtype=int)

# Storage
rom_images = []
titles = []

for i, name in enumerate(display_waveforms):
    r = results[name]
    ax = axes[i]
    
    vmax = np.percentile(np.abs(r['x_rom'][:max_idx] @ Phi[N:, :].T), 95) + 1e-10
    
    p_rom_0 = (Phi @ r['x_rom'][0, :])[N:].reshape(Nx, Nz).T
    
    im = ax.imshow(p_rom_0, aspect='equal', cmap='RdBu_r',
                   vmin=-vmax, vmax=vmax, extent=[0, Lx, Lz, 0])
    plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04, label='Pa')
    ax.set_xlabel('Range (m)')
    if i == 0:
        ax.set_ylabel('Depth (m)')
    
    status = '✓' if r['error'] < 5 else '⚠' if r['error'] < 20 else '✗'
    title = ax.set_title(f"{name}\n{status} Err: {r['error']:.1f}% | t=0ms", fontsize=10)
    
    rom_images.append(im)
    titles.append(title)

# Slider and button
ax_slider = plt.axes([0.15, 0.05, 0.5, 0.03])
ax_button = plt.axes([0.7, 0.05, 0.1, 0.03])

slider = Slider(ax_slider, 'Frame', 0, n_frames - 1, valinit=0, valstep=1)
button = Button(ax_button, 'Play')

anim_running = [False]
current_frame = [0]

def update_frame(frame_num):
    frame_num = int(frame_num)
    idx = frame_indices[frame_num]
    t_ms = t_rom[idx] * 1000
    
    for i, name in enumerate(display_waveforms):
        r = results[name]
        p_rom = (Phi @ r['x_rom'][idx, :])[N:].reshape(Nx, Nz).T
        rom_images[i].set_data(p_rom)
        
        status = '✓' if r['error'] < 5 else '⚠' if r['error'] < 20 else '✗'
        titles[i].set_text(f"{name}\n{status} Err: {r['error']:.1f}% | t={t_ms:.0f}ms")
    
    fig.canvas.draw_idle()

def on_slider_change(val):
    current_frame[0] = int(val)
    update_frame(val)

def animate():
    if anim_running[0]:
        current_frame[0] = (current_frame[0] + 1) % n_frames
        slider.set_val(current_frame[0])
        timer.start(50)

def on_button_click(event):
    if anim_running[0]:
        anim_running[0] = False
        button.label.set_text('Play')
        timer.stop()
    else:
        anim_running[0] = True
        button.label.set_text('Pause')
        animate()

slider.on_changed(on_slider_change)
button.on_clicked(on_button_click)

timer = fig.canvas.new_timer(interval=50)
timer.add_callback(animate)

plt.suptitle(f'ROM (q={q_pod}) Response to Different Sources (0-800ms)', fontsize=14)
plt.show()